In [ ]:

%load_ext autoreload
%autoreload 2

import os

import pytesseract
from pytesseract import Output
import cv2

# Libraries for Visualization
from PIL import Image, ImageDraw
from IPython.display import display

import src.wordbox
from src.graph import construct_graph

In [ ]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [ ]:
def annotate_image(filepath, boxes):
# Visualize the Box and Neighbours using PIL
    kWidth = 2
    kBoxColor = (0, 0, 255, 255)
    with Image.open(filepath).convert('RGBA') as img:
        draw = ImageDraw.Draw(img)
    
        for box in boxes:
            top, bottom, left, right = box.top, box.top + box.height, box.left, box.left + box.width
            # Top, Bottom, Left, Right Lines
            draw.line((left, top, right, top), width=kWidth, fill=kBoxColor)
            draw.line((left, bottom, right, bottom), width=kWidth, fill=kBoxColor)
            draw.line((left, top, left, bottom), width=kWidth, fill=kBoxColor)
            draw.line((right, top, right, bottom), width=kWidth, fill=kBoxColor)

            # Plot edges.
            for name, neighbour in box.neighs.items():
                if neighbour is not None:
                    neigh = neighbour['which']
                    if name == 'top':
                        boxPos = (box.left + box.width / 2, box.top)
                        neighPos = (neigh.left + neigh.width / 2, neigh.top + neigh.height)
                    elif name == 'bottom':
                        boxPos = (box.left + box.width / 2, box.top + box.height)
                        neighPos = (neigh.left + neigh.width / 2, neigh.top)
                    elif name == 'left':
                        boxPos = (box.left, box.top + box.height / 2)
                        neighPos = (neigh.left + neigh.width, neigh.top + neigh.height / 2)
                    elif name == 'right':
                        boxPos = (box.left + box.width, box.top + box.height / 2)
                        neighPos = (neigh.left, neigh.top + neigh.height / 2)
                    # Draw Line between Gravity Center of Box and Neighbour
                    draw.line(boxPos + neighPos, width=kWidth, fill=(0, 255, 0, 255))
        display(img)

In [ ]:
boxes = construct_graph('./img/test.jpg')
print('\n'.join([str(elem) for elem in boxes]))
annotate_image('./img/test.jpg', boxes)